In [ ]:
import cv2
import os
from ultralytics import YOLO

# Model yolu
model = YOLO(r"../pt/drone_best.pt")
classes_to_count = [0]

def process_video():
    video_path = r"C:\Users\524ha\Desktop\AYGAZ_DATAS\Datasets\yarımca_kamyon\Annotation\unlabeled\Yandan_0508\secondary_labeling\images"
    destination_folder = r"C:\Users\524ha\Desktop\AYGAZ_DATAS\Datasets\yarımca_kamyon\Annotation\unlabeled\Yandan_0508\secondary_labeling\images\labels"

    labels_dir = os.path.join(destination_folder, "labels")
    annotated_dir = os.path.join(destination_folder, "annotated")
    images_dir = os.path.join(destination_folder, "images")  # çizimsiz kareler

    os.makedirs(labels_dir, exist_ok=True)
    os.makedirs(annotated_dir, exist_ok=True)
    os.makedirs(images_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), f"Video açılamadı: {video_path}"

    frame_count = 0
    saved = 0
    while cap.isOpened():
        success, im0 = cap.read()
        if not success:
            break

        frame_count += 1
        # Her 4 karede 1 işle
        if frame_count % 4 != 0:
            continue

        results = model.predict(im0, verbose=False, classes=classes_to_count)

        # Etiket yoksa hiçbir şey kaydetme
        if len(results[0].boxes) == 0:
            continue

        frame_name = f"frame_{frame_count}"
        txt_path = os.path.join(labels_dir, f"{frame_name}.txt")
        annotated_path = os.path.join(annotated_dir, f"{frame_name}.jpg")
        raw_path = os.path.join(images_dir, f"{frame_name}.jpg")

        # TXT yaz
        with open(txt_path, 'w') as txt_file:
            for box in results[0].boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                class_id = int(box.cls)

                width = x2 - x1
                height = y2 - y1
                center_x = x1 + width / 2
                center_y = y1 + height / 2
                img_height, img_width, _ = im0.shape

                yolo_format = f"{class_id} {center_x / img_width:.6f} {center_y / img_height:.6f} {width / img_width:.6f} {height / img_height:.6f}\n"
                txt_file.write(yolo_format)

        # Çizimsiz kare
        cv2.imwrite(raw_path, im0)
        # Çizimli kare
        annotated_img = results[0].plot()
        cv2.imwrite(annotated_path, annotated_img)

        saved += 1

    cap.release()
    print(f"Bitti ✅ Kaydedilen kare sayısı: {saved}")

# Çalıştır
process_video()


Bitti ✅ Kaydedilen kare sayısı: 656
